## ANALYSIS - CASE STUDY STATISTA

In [55]:
import pandas as pd
import yfinance as yf 
import numpy as np

# 1. Read tickers

In [ ]:
#data downloaded from github
my_tickers = pd.read_csv("data/input/constituents.csv")
my_tickers.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [57]:
my_tickers.shape

(503, 8)

In [58]:
#use 200 of them
my_tickers = my_tickers[:200]

# 2. Clean data

In [59]:
duplicated = my_tickers[my_tickers['Security'].str.contains('Class')]
print(duplicated)

   Symbol                 Security             GICS Sector  \
19  GOOGL  Alphabet Inc. (Class A)  Communication Services   
20   GOOG  Alphabet Inc. (Class C)  Communication Services   

               GICS Sub-Industry      Headquarters Location  Date added  \
19  Interactive Media & Services  Mountain View, California  2014-04-03   
20  Interactive Media & Services  Mountain View, California  2006-04-03   

        CIK Founded  
19  1652044    1998  
20  1652044    1998  


In [60]:
my_tickers = my_tickers.drop([20])

In [61]:
#Check for empties
my_tickers.isnull().sum()

Symbol                   0
Security                 0
GICS Sector              0
GICS Sub-Industry        0
Headquarters Location    0
Date added               0
CIK                      0
Founded                  0
dtype: int64

In [62]:
my_tickers.shape

(199, 8)

# 3. Extract data

In [63]:
# extract tickers to use
tickers_list = my_tickers['Symbol'].tolist()
print(tickers_list[:5])


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [64]:
# Convert the list of tickers to enable data extraction
tickers_list = [yf.Ticker(ticker) for ticker in tickers_list]
#print(tickers_list[:5])

In [98]:
def company_data(tickers_list):
    row_data = []

    for ticker in tickers_list:
        info = ticker.info

        try:
            finance = ticker.financials
            balance = ticker.balance_sheet

            # Fetch last 3 years
            def last_three(df, metric):
                #most data for 2025 is not available
                allowed = {2024,2023,2022} 
                if df is None or metric not in df.index:
                    return[]
                
                #sort
                cols = sorted(df.columns, reverse=True)

                results = []
                for col in cols[:3]:
                    year = col.year
                    if year not in allowed:
                        continue
                    val = df.loc[metric, col]
                    results.append((year, val / 1e6 if val is not None else None))

                    if len(results) == 3:
                        break
                return results
            
            rev_yrs = last_three(finance, "Total Revenue")
            net_yrs = last_three(finance, "Net Income")
            assets_yrs = last_three(balance, "Total Assets")
                


            row ={
                "Ticker":info.get("symbol"),
                "Company name": info.get("shortName") or info.get("longName"),
                "Country":info.get("country"),
                "Industry":info.get("industry"),
                "Revenue unit": info.get("currency"),
                "Market Cap (millions)":info.get("marketCap")/ 1e6 if info.get("marketCap") is not None else None
            }

            for year, value in rev_yrs:
                row[f"Revenue_{year} (millions)"] = value

            for year, value in net_yrs:
                row[f"NetIncome_{year} (millions)"] = value

            for year, value in assets_yrs:
                row[f"TotalAssets_{year} (millions)"] = value

            row_data.append(row) 

        except Exception as e:
            print(f"error processing ticker{ticker}")
        
            row_data.append({
            "Ticker":info.get("symbol"),
            "Company name": None,
            "Country":None,
            "Industry":None,
            "Revenue unit":None,
            "Revenue_Y1 (millions)":None,"Revenue_Y2 (millions)": None, "Revenue_Y3 (millions)": None,
            "Net Income_Y1 (millions)": None,"NetIncome_Y2 (millions)": None, "NetIncome_Y3 (millions)": None,
            "Total Assets_Y1 (millions)": None, "TotalAssets_Y2 (millions)": None, "TotalAssets_Y3 (millions)": None,
            "Market Cap":None
        })

    my_data = pd.DataFrame(row_data)
    #round up for readabilityu
    numeric_cols = [c for c in my_data.columns if "millions" in c]
    my_data[numeric_cols] = my_data[numeric_cols].astype(float).round(2)


    return my_data

company_data(tickers_list)

my_data = company_data(tickers_list)



In [99]:
# check for missing values
my_data.isnull().sum()

Ticker                          0
Company name                    2
Country                         2
Industry                        2
Revenue unit                    2
Market Cap (millions)           2
Revenue_2024 (millions)         2
Revenue_2023 (millions)         2
Revenue_2022 (millions)        42
NetIncome_2024 (millions)       2
NetIncome_2023 (millions)       2
NetIncome_2022 (millions)      42
TotalAssets_2024 (millions)     2
TotalAssets_2023 (millions)     2
TotalAssets_2022 (millions)    40
dtype: int64

In [100]:
my_data[my_data.isnull().any(axis=1)]

,Ticker,Company name,Country,Industry,Revenue unit,Market Cap (millions),Revenue_2024 (millions),Revenue_2023 (millions),Revenue_2022 (millions),NetIncome_2024 (millions),NetIncome_2023 (millions),NetIncome_2022 (millions),TotalAssets_2024 (millions),TotalAssets_2023 (millions),TotalAssets_2022 (millions)
4,ACN,Accenture plc,Ireland,Information Technology Services,USD,155713.18,64896.46,64111.74,NaN,7264.79,6871.56,NaN,55932.36,51245.30,NaN
10,APD,"Air Products and Chemicals, Inc",United States,Specialty Chemicals,USD,58107.18,12100.60,12600.00,NaN,3828.20,2300.20,NaN,39574.60,32002.50,NaN
22,AMCR,Amcor plc,Switzerland,Packaging & Containers,USD,19667.23,13640.00,14694.00,NaN,730.00,1048.00,NaN,16524.00,17003.00,NaN
33,ADI,"Analog Devices, Inc.",United States,Semiconductors,USD,129924.82,9427.16,12305.54,NaN,1635.27,3314.58,NaN,48228.28,48794.48,NaN
37,AAPL,Apple Inc.,United States,Consumer Electronics,USD,4138242.67,391035.00,383285.00,NaN,93736.00,96995.00,NaN,364980.00,352583.00,NaN
38,AMAT,"Applied Materials, Inc.",United States,Semiconductor Equipment & Materials,USD,200953.05,27176.00,26517.00,NaN,7177.00,6856.00,NaN,34409.00,30729.00,26726.0
46,ATO,Atmos Energy Corporation,United States,Utilities - Regulated Gas,USD,28517.86,4165.19,4275.36,NaN,1042.90,885.86,NaN,25194.46,22516.97,NaN
47,ADSK,"Autodesk, Inc.",United States,Software - Application,USD,64611.42,5497.00,5005.00,NaN,906.00,823.00,NaN,9912.00,9438.00,NaN
48,ADP,"Automatic Data Processing, Inc.",United States,Software - Application,USD,103419.36,19202.60,18012.20,NaN,3752.00,3412.00,NaN,54362.70,50971.00,NaN
49,AZO,"AutoZone, Inc.",United States,Auto Parts,USD,65898.91,18490.27,17457.21,NaN,2662.43,2528.43,NaN,17176.54,15985.88,NaN


In [102]:
my_data[my_data["Company name"].isna()]

,Ticker,Company name,Country,Industry,Revenue unit,Market Cap (millions),Revenue_2024 (millions),Revenue_2023 (millions),Revenue_2022 (millions),NetIncome_2024 (millions),NetIncome_2023 (millions),NetIncome_2022 (millions),TotalAssets_2024 (millions),TotalAssets_2023 (millions),TotalAssets_2022 (millions)
58,BRK.B,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,BF.B,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
my_data = my_data.dropna()

In [108]:
my_data.isnull().sum()

Ticker                         0
Company name                   0
Country                        0
Industry                       0
Revenue unit                   0
Market Cap (millions)          0
Revenue_2024 (millions)        0
Revenue_2023 (millions)        0
Revenue_2022 (millions)        0
NetIncome_2024 (millions)      0
NetIncome_2023 (millions)      0
NetIncome_2022 (millions)      0
TotalAssets_2024 (millions)    0
TotalAssets_2023 (millions)    0
TotalAssets_2022 (millions)    0
dtype: int64

In [109]:
my_data.shape

(157, 15)

In [ ]:
%pip install XlsxWriter


In [112]:
excel_file = "data/Case_study_output.xlsx"

my_data.to_excel(excel_file, index=False, sheet_name="Financials")

print(f"Saved Excel file {excel_file}")

    

Saved Excel file data/Case_study_output.xlsx


In [113]:
my_data.to_csv("data/Case_study_output.csv", index=False)
print("Saved data to my_data.csv")

Saved data to my_data.csv
